In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image


classificator = Sequential()
classificator.add(Conv2D(32, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
classificator.add(BatchNormalization())
classificator.add(MaxPooling2D(pool_size = (2,2)))

classificator.add(Conv2D(32, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
classificator.add(BatchNormalization())
classificator.add(MaxPooling2D(pool_size = (2,2)))

classificator.add(Conv2D(32, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
classificator.add(BatchNormalization())
classificator.add(MaxPooling2D(pool_size = (2,2)))


classificator.add(Flatten())

classificator.add(Dense(units = 256, activation = 'relu'))
classificator.add(Dropout(0.3))
classificator.add(Dense(units = 128, activation = 'relu'))
classificator.add(Dropout(0.3))
classificator.add(Dense(units = 64, activation = 'relu'))
classificator.add(Dropout(0.3))
classificator.add(Dense(units = 1, activation = 'sigmoid'))

classificator.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                      metrics = ['accuracy'])

training_generator = ImageDataGenerator(rescale = 1./255)
                                        #  rotation_range = 7,
                                        #  horizontal_flip = True,
                                        #  shear_range = 0.2,
                                        #  height_shift_range = 0.07,
                                        #  zoom_range = 0.2)
                                         
test_generator = ImageDataGenerator(rescale = 1./255)

2021-11-08 08:15:16.474027: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-08 08:15:16.474046: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-11-08 08:15:17.708555: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-08 08:15:17.708591: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-08 08:15:17.708625: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (israel-Nitro-AN515-51): /proc/driver/nvidia/version does not exist
2021-11-08 08:15:17.708819: I tensorflow/core/platform/

In [6]:
batch_size = 30
training_base = training_generator.flow_from_directory('capela/dataset/training',
                                                           target_size = (64, 64),
                                                           batch_size = batch_size,
                                                           class_mode = 'binary')

test_base = test_generator.flow_from_directory('capela/dataset/validation',                                               
                                               target_size = (64, 64),
                                               batch_size = batch_size,
                                               class_mode = 'binary')
print(training_base.class_indices)
print(test_base.class_indices)

Found 598 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
{'capela_anomalia': 0, 'capela_normal': 1}
{'capela_anomalia': 0, 'capela_normal': 1}


In [6]:
classificator.fit_generator(training_base, steps_per_epoch = 10,
                            epochs = 100, validation_data = test_base,
                            validation_steps = 10)

# 

/home/israel/.local/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-11-08 08:11:10.038560: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
10/10 [==============================] - 3s 271ms/step - loss: 0.9871 - accuracy: 0.5705 - val_loss: 0.6937 - val_accuracy: 0.5000
Epoch 2/100
10/10 [==============================] - 1s 144ms/step - loss: 0.9805 - accuracy: 0.5000
Epoch 3/100
10/10 [==============================] - 1s 144ms/step - loss: 0.8183 - accuracy: 0.5067
Epoch 4/100
10/10 [==============================] - 1s 142ms/step - loss: 0.6765 - accuracy: 0.6000
Epoch 5/100
10/10 [==============================] - 2s 169ms/step - loss: 0.5941 - accuracy: 0.7167
Epoch 6/100
10/10 [==============================] - 2s 164ms/step - loss: 0.3811 - accuracy: 0.8300
Epoch 7/100
10/10 [==============================] - 2s 150ms/step - loss: 0.1945 - accuracy: 0.9329
Epoch 8/100
10/10 [==============================] - 2s 158ms/step - loss: 0.0786 - accuracy: 0.9667
Epoch 9/100
10/10 [==============================] - 2s 164ms/step - loss: 0.0840 - accuracy: 0.9767
Epoch 10/100
10/10 [=============================

In [7]:
classificator.save('capela/weights/weight.h5')

In [3]:
classificator.load_weights('capela/weights/weight.h5')

In [7]:
imagem_teste = image.load_img('capela/dataset/test/capela_anomalia/capela_anomalia.105.png',
                              target_size = (64,64))

imagem_teste = image.img_to_array(imagem_teste)
imagem_teste /= 255
imagem_teste = np.expand_dims(imagem_teste, axis = 0)
previsao = classificator.predict(imagem_teste)
print(previsao)
previsao = (previsao > 0.5)
print(previsao)
training_base.class_indices


[[1.5312278e-10]]
[[False]]


{'capela_anomalia': 0, 'capela_normal': 1}

In [26]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
validation_generator1 = ImageDataGenerator(rescale = 1./255)
base_validation1 = validation = validation_generator1.flow_from_directory('capela/dataset/test/',
                                               target_size = (64, 64),
                                               batch_size = 10,
                                               class_mode = 'binary')

y_pred = classificator.predict_generator(base_validation1, steps = 10)
base_validation1.class_indices

validation_generator2 = ImageDataGenerator(rescale = 1./255)
base_validation2 = validation = validation_generator2.flow_from_directory('capela/dataset/test2/',
                                               target_size = (64, 64),
                                               batch_size = 10,
                                               class_mode = 'binary')

y_pred2 = classificator.predict_generator(base_validation2, steps = 10)
base_validation2.class_indices
# {'capela_anomalia': 0, 'capela_normal': 1}

Found 100 images belonging to 2 classes.


/home/israel/.local/lib/python3.8/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Found 100 images belonging to 2 classes.


{'capela_anomalia': 0, 'capela_normal': 1}

In [ ]:
y_pred = np.squeeze(np.asarray(y_pred))
for x in y_pred:
    print(x)

In [32]:
# [50]

y_pred = np.around(np.squeeze(np.asarray(y_pred)))
y_pred2 = np.around(np.squeeze(np.asarray(y_pred2)))

# y_pred = np.squeeze(np.asarray(y_pred))
# y_pred2 = np.squeeze(np.asarray(y_pred2))

y_p = np.concatenate((y_pred, y_pred2))
y_classes = np.concatenate((base_validation1.classes, base_validation2.classes))

print(y_classes)
print(y_p)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.

In [33]:

matrix = confusion_matrix(y_classes, y_p)
print(matrix)

[[100   0]
 [  0 100]]


In [34]:
report = metrics.classification_report(y_classes, y_p, target_names=base_validation1.class_indices)
print(report)    


                 precision    recall  f1-score   support

capela_anomalia       1.00      1.00      1.00       100
  capela_normal       1.00      1.00      1.00       100

       accuracy                           1.00       200
      macro avg       1.00      1.00      1.00       200
   weighted avg       1.00      1.00      1.00       200

